In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

In [32]:
a = -2
tf.abs(a)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [33]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0, 9.0], dtype=float)
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
history = model.fit(xs, ys, epochs=500, verbose=0)

In [34]:
inp = float(input('please enter the desired input for output prediction: '))
print("predicted_output: ", np.round(model.predict([inp])[0][0]))

please enter the desired input for output prediction: 


ValueError: could not convert string to float: ''

In [ ]:
def huber_loss(y_true, y_pred):
    error = y_true - y_pred
    delta_thresh = 1
    is_small = tf.abs(error) <= delta_thresh
    small_error = 0.5*tf.square(error)
    big_error = delta_thresh*(tf.abs(error) - 0.5*delta_thresh)
    return tf.where(is_small, small_error, big_error)

In [ ]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=huber_loss)
history = model.fit(xs, ys, epochs=500, verbose=0)

In [ ]:
inp = float(input('please enter the desired input for output prediction: '))
print("predicted_output: ", np.round(model.predict([inp])[0][0]))

In [ ]:
plt.plot(history.history['loss'])

## How to create huber loss function with wrapper function

In [35]:
def huber_loss_with_threshold(threshold):
    def huber_loss(y_true, y_pred):
        error = y_true - y_pred
        threshold = 1
        is_small = tf.abs(error) <= threshold
        small_error = 0.5*tf.square(error)
        big_error = threshold*(tf.abs(error) - 0.5*threshold)
        return tf.where(is_small, small_error, big_error)
    return huber_loss

In [36]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=huber_loss_with_threshold(threshold=1))
history = model.fit(xs, ys, epochs=500, verbose=0)

In [37]:
inp = float(input('please enter the desired input for output prediction: '))
print("predicted_output: ", np.round(model.predict([inp])[0][0]))

please enter the desired input for output prediction: !@


ValueError: could not convert string to float: '!@'

## How to make huber loss function class

In [38]:
from tensorflow.keras.losses import Loss
class HuberLoss(Loss):
    threshold = 1
    def __init__(self, threshold):
        super().__init__()
        self.threshold = threshold
    
    def call(self, y_true, y_pred):        
        error = y_true - y_pred        
        is_small_error = tf.abs(error) <= self.threshold
        small_error = 0.5*tf.square(error)
        big_error = self.threshold*(tf.abs(error) - 0.5*self.threshold)
        return tf.where(is_small_error, small_error, big_error)
    

In [39]:
model = tf.keras.Sequential([
    keras.layers.Dense(units=1, input_shape=[1])
])
model.compile(optimizer='sgd',
              loss=HuberLoss(threshold=1))
history = model.fit(xs, ys, 
                    epochs=500, verbose=0)

## Contrastive Loss function

In [40]:
from tensorflow.keras import backend as K
def contrastive_loss(y_true, y_pred):
    margin = 1
    squared_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_pred*squared_pred + (1-y_true)*margin_square)

In [42]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0, 9.0], dtype=float)

model = tf.keras.Sequential([
    keras.layers.Dense(units=1, input_shape=[1])
])
model.compile(optimizer='sgd',
              loss=contrastive_loss)
history = model.fit(xs, ys, 
                    epochs=500, verbose=0)

In [43]:
model.predict([6.0])

array([[nan]], dtype=float32)